#Short Programming Task

Suppose you are writing software for an art installation which involves a chandelier which can be moved up and down by a motor. The input to your program will be a CSV file called schedule.csv with hourly timestamps (in 'YYYY-MM-DDTHH:MM:SSZ' format) and an integer value representing the desired height of the chandelier for a period of one hour starting at that timestamp. For example:  
2025-01-01T01:00:00Z, 10
2025-01-01T02:00:00Z, 15
2025-01-01T03:00:00Z, 15
2025-01-01T04:00:00Z, 13
...
2025-01-04T05:00:00Z, 5
2025-01-04T06:00:00Z, 3

The motor can either be stationary or it can move at a constant speed of 1 metre per minute. It will change between the hourly steps symmetrically about the hour boundary, i.e. half of its movement will be before the hour boundary and half will be afterward.

Your software needs to write a file called output.csv which contains the minimal sequence of points that, when linearly interpolated, describe the actual height of the chandelier over time after accounting for the movement of the motor.

For the example above, the output.csv should be:
2025-01-01T01:00:00Z, 10
2025-01-01T01:57:30Z, 10
2025-01-01T02:02:30Z, 15
2025-01-01T03:59:00Z, 15
2025-01-01T04:01:00Z, 13
...
2025-01-04T05:59:00Z, 5
2025-01-04T06:01:00Z, 3
2025-01-04T07:00:00Z, 3



In [ ]:
import csv
import pytest
from datetime import datetime, timedelta

def read_file(filePath):
    schedule = []
    with open(filePath, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            timestamp, height = row
            schedule.append((datetime.fromisoformat(timestamp.rstrip('Z')), int(height)))
    return schedule

def write_output(filePath, data):
    with open(filePath, 'w', newline='') as file:
        writer = csv.writer(file)
        for row in data:
            writer.writerow([row[0].isoformat() + 'Z', row[1]])

def process_chandelier_data(filePath = "schedule.csv", output_FilePath = "output.csv"):
    schedule = read_file(filePath)
    points = []
    for i in range(len(schedule)):
        timestamp, height = schedule[i]
        if i == 0:
            points.append((timestamp, height))
        
        if i < len(schedule) - 1:
            next_timestamp, next_height = schedule[i + 1]
            if height != next_height:
                height_difference = abs(next_height - height)
                midpoint_height = height_difference / 2
                transition_start = next_timestamp - timedelta(minutes=midpoint_height)
                transition_end = next_timestamp + timedelta(minutes=midpoint_height)
                points.append((transition_start, height))
                points.append((transition_end, next_height))
    
    final_point = schedule[-1]
    points.append((final_point[0] + timedelta(hours=1), final_point[1]))
    write_output(output_FilePath, points)

def test_chandelier_output(test_output_filePath):
    process_chandelier_data()
    output_data = read_file("output.csv")
    expected_output_data = read_file(test_output_filePath)
    assert output_data == expected_output_data

if __name__ == '__main__':
    test_chandelier_output("test_output_file.csv")